In [1]:
import sys
sys.path.append("C:\\Users\\lauth\\OneDrive\\Desktop\\sql_assistant_v3")

In [13]:
# Modelos LLM
from src.components.models.embeddings.embeddings import HF_MultilingualE5_Embeddings, Openai_Embeddings
from src.components.models.llms.llms import HF_Llama38b_LLM, Openai_LLM


hf_llm = HF_Llama38b_LLM()
openai_llm = Openai_LLM()

# Modelos Embeddings
mle5_embeddings = HF_MultilingualE5_Embeddings()
openai_embeddings = Openai_Embeddings()

#Iniciando los modelos
openai_llm.init_model()
hf_llm.init_model()
mle5_embeddings.init_model()
openai_embeddings.init_model()

In [14]:
from src.db.chroma_db.handlers.handlers import (
    add_base_columns,
    melt_columns,
    process_searched_columns,
    process_searched_relations,
    query_by_vector_embedding,
)
from src.settings.settings import Settings
from experiments.experiments_settings.settings import Experiments_Settings
from src.components.models.models_interfaces import Base_Embeddings
from src.utils.utils import clean_sentence


def retrieve_sql_semantic_information_improved(
    keywords_arr: list[str],
    embeddings: Base_Embeddings,
    relations_search_properties: dict[str, any] = {"n": 5, "score_threshold": 0.7},
    columns_search_properties: dict[str, any] = {"n": 4, "score_threshold": 0.6},
):
    """ """
    vectors = []
    relations_collection = Experiments_Settings.Chroma.get_experiments_semantic_relations_collection()
    columns_collection = Experiments_Settings.Chroma.get_experiments_semantic_columns()
    
    tables = set()
    results_columns_collection = tuple()
    combined_dict = {}
    tables_related = [] # Arreglo de tablas con columnas necesarias para JOINS 
    table_relations_descriptions = set()
    
    # Parte 0: Vectorizando las palabras
    for kw_item in keywords_arr:
        v = embeddings.get_embeddings(clean_sentence(kw_item))
        vectors.append(v)
    
    # Parte 1: Buscamos las tablas
    for _, kw_vector in enumerate(vectors):
        # Buscamos en la db
        results_relations_collection = query_by_vector_embedding(
            collection=relations_collection,
            vector_embedding=kw_vector,
            n=relations_search_properties["n"],
            score_threshold=relations_search_properties["score_threshold"],
            metadata_filters={},
        )
        if not results_relations_collection:
            continue

        # Formateamos la data recuperada
        if len(results_relations_collection) > 0:
            x = process_searched_relations(items=results_relations_collection, tables_related={})
            tables_related.append(x["tables_related"]) 
            table_relations_descriptions.update(x["table_relations_descriptions"]) 
            
            tables = set(x["tables_related"].keys()).union(tables)
            
    # Parte 2: Busqueda semantica de columnas (porsiacaso una extra jeje)
    for _, kw_vector in enumerate(vectors):
        for table in tables:
            results_columns_collection += query_by_vector_embedding(
                collection=columns_collection,
                vector_embedding=kw_vector,
                n=columns_search_properties["n"],
                score_threshold=columns_search_properties["score_threshold"],
                metadata_filters={"meta_table": {"$eq": table}},
            )        
            if len(results_columns_collection) > 0:
                current_data = process_searched_columns(results_columns_collection, {}, 0)
                combined_dict.update(add_base_columns(tables, current_data))
            else:
                combined_dict.update(add_base_columns(tables, {}))
        
    # Parte 3: Merge de las columnas normales y las columnas para JOINS
    resultado_columnas = melt_columns(tables_related, combined_dict)
    
    return (
        tables,
        resultado_columnas,
        (
           table_relations_descriptions
            if table_relations_descriptions
            else []
        ),
    )
    

In [17]:
request_2 = ['equipment type classification', 'measurement data', 'measurement system', 'firmware', 'variable']

r_2 = retrieve_sql_semantic_information_improved(request_2, openai_embeddings)

In [18]:
r_2[0]

{'equ_equipo',
 'fcs_computador_medidor',
 'fcs_computadores',
 'fcs_firmware',
 'fcs_tipo_computador',
 'flu_tipo_fluido',
 'med_sistema_medicion',
 'med_tag',
 'med_tipo_medicion',
 'pla_plataforma',
 'teq_clasificacion',
 'teq_tipo_equipo',
 'var_tipo_variable',
 'var_variable_datos'}

In [6]:
from src.utils.reader_utils import read_database_semantics, read_tables_data

create_table_template = """CREATE TABLE IF NOT EXISTS dbo_v2.{table_name}(\n{list_columns_plus_type_plus_descriptions});"""

def _add_ddl_in_prompt(semantic_info: dict[str, any]):
    ddls = list()
    ddl_text = ""
    
    for table in semantic_info:
        list_columns_plus_type_plus_descriptions = set()
        for item in semantic_info[table]:
            elementos = [item[4]] if isinstance(item[4], str) else list(item[4])
            if all(elemento in semantic_info for elemento in elementos):
                list_columns_plus_type_plus_descriptions.add(
                    (f"\t{item[0]} {item[1]} -- Description: {item[2]},\n", item[3])
                )
        # Crear una lista ordenada de las columnas basadas en la prioridad
        ordered_list = sorted(
            list_columns_plus_type_plus_descriptions, key=lambda x: x[1]
        )
        list_columns_plus_type_plus_descriptions = "".join(
            tupla[0] for tupla in ordered_list
        )
        t = create_table_template.format(
            table_name=table,
            list_columns_plus_type_plus_descriptions=list_columns_plus_type_plus_descriptions,
        )
        ddls.append(t)

    tables_context = "\n".join(ddls)
    ddl_text += f"\n{tables_context}\n\n"
    
    rel = read_database_semantics(
        "relations", ["table_1", "table_2", "join_description"]
    )
    for _, item in rel.iterrows():
        table_1_plus_table_2 = [item["table_1"], item["table_2"]]
        if all(elemento in semantic_info for elemento in table_1_plus_table_2):
            ddl_text += f"""{item["join_description"]}\n"""

    return ddl_text

In [7]:
a = _add_ddl_in_prompt(r_2[1])
print(a)


CREATE TABLE IF NOT EXISTS dbo_v2.pla_plataforma(
	Id INT -- Description: Unique Identifier for each platform,
	Nombre VARCHAR(MAX) -- Description: Platform name,
	Estado VARCHAR(10) -- Description: Platform status, active ('ACT') or inactive ('INA'),
);
CREATE TABLE IF NOT EXISTS dbo_v2.flu_tipo_fluido(
	Id INT -- Description: Unique identifier for the fluid type,
	Nombre VARCHAR(100) -- Description: Name of the fluid type, must be in: (Gás Natural / Óleo Cru / Água / Condensado / Vapour / Água + MEG / MEG),
	Estado VARCHAR(100) -- Description: Fluid type status, active ('ACT') or inactive ('INA'),
);
CREATE TABLE IF NOT EXISTS dbo_v2.med_tipo_medicion(
	Id INT -- Description: Unique identifier for each measure type,
	Nombre VARCHAR(150) -- Description: Name of the measurement type,
	Estado VARCHAR(10) -- Description: Status of the measurement type, active ('ACT') or inactive ('INA'),
);
CREATE TABLE IF NOT EXISTS dbo_v2.med_sistema_medicion(
	Id INT -- Description: Unique identifier